<a href="https://colab.research.google.com/github/rgolds5/DS-Unit-1-Sprint-2-Data-Wrangling-and-Storytelling/blob/master/Gapminder_GIF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
:
###Do not run this in jupyter..it will likely crash. Implemented in Visual Studio Code



# Importing in necessary libraries
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import os

sns.set_style('white')


csv_dict = {
    'income': 'https://raw.githubusercontent.com/open-numbers/ddf--gapminder--systema_globalis/master/ddf--datapoints--income_per_person_gdppercapita_ppp_inflation_adjusted--by--geo--time.csv',
    'lifespan': 'https://raw.githubusercontent.com/open-numbers/ddf--gapminder--systema_globalis/master/ddf--datapoints--life_expectancy_years--by--geo--time.csv',
    'population': 'https://raw.githubusercontent.com/open-numbers/ddf--gapminder--systema_globalis/master/ddf--datapoints--population_total--by--geo--time.csv',
    'entities': 'https://raw.githubusercontent.com/open-numbers/ddf--gapminder--systema_globalis/master/ddf--entities--geo--country.csv',
    'concepts': 'https://raw.githubusercontent.com/open-numbers/ddf--gapminder--systema_globalis/master/ddf--concepts.csv',
}

dfs = {}

for k in csv_dict:
    dfs[k] = pd.read_csv(csv_dict[k])

df = (dfs['income']
    .merge(dfs['lifespan'])
    .merge(dfs['population'])
    .merge(dfs['entities'][['country', 'name', 'world_6region']], 
        left_on = 'geo', right_on = 'country')
    .drop(columns = ['geo', 'country'])
    .rename(columns = {'time': 'year', 
                'income_per_person_gdppercapita_ppp_inflation_adjusted': 'income',
                'life_expectancy_years': 'lifespan',
                'name': 'country',
                'population_total': 'population',
                'world_6region': 'region'}))

#Loop that creates a chart for each year..for this set it creates 200+ separate 
#charts, so can be a data overload...but it can be filtered down to smaller number
#of charts.

for year in df['year'].unique():
    tmp = df[df['year'] == year]
    sns.relplot(x = 'income',
            y = 'lifespan',
            hue = 'region',
            size = 'population',
            data = tmp,
            sizes = (5,1000),
            height = 5,
            aspect = 1.5)
  plt.xscale('log')
  plt.xlabel('Income Per Person')
  plt.ylabel('Average Lifespan')
  plt.title(f'Year: {str(year)}')
  plt.xlim(10, 1300000)
  plt.ylim(0, 100)
  filename = f'Gapminderstep_{year}.png'
  plt.savefig(filename, dpi = 96)
  plt.gca()



#ImageMagick Implementation

gif_name = 'animated_gapminder'
file_list = glob.glob('*.png')
list.sort(file_list, key = lambda x: int(x.split('_')[1].split('.png')[0]))

with open('image_list.txt', 'w') as file:
  for item in file_list:
    file.write('%s\n' % item)

os.system(f'magick -delay 1x10 @image_list.txt {gif_name}.gif')